# Go through directories and sort by brightness to make cleaning easier

In [1]:
%matplotlib inline
#from pylab import rcParams
#rcParams['figure.figsize'] = 12, 8

%load_ext autoreload
%autoreload 2

%load_ext version_information
%version_information numpy, scipy, matplotlib, pandas

Software versions
Python 2.7.13 64bit [GCC 4.2.1 Compatible Apple LLVM 8.0.0 (clang-800.0.42.1)]
IPython 5.4.1
OS Darwin 16.6.0 x86_64 i386 64bit
numpy 1.13.0
scipy 0.19.1
matplotlib 2.0.2
pandas 0.20.2
Mon Jul 10 10:35:34 2017 AEST

In [2]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import cv2
import glob
import copy

import os
import sys
import glob
import hashlib
import uuid
from sklearn.metrics import mean_squared_error

#sys.path.append("../src")
#from images_permute import processing_pipeline

np.random.seed(0)

def view(image):
    plt.figure()
    plt.axis('off')
    plt.imshow(image, cmap = cm.Greys_r)

## Sort by pixel total

In [4]:
def mse(imageA, imageB):
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err

In [20]:
list_image_directory_paths = glob.glob('/Users/jacksimpson/Data/beeunique/output/drive8-merged/*/')
print(list_image_directory_paths[0], len(list_image_directory_paths))

('/Users/jacksimpson/Data/beeunique/output/drive8-merged/0/', 54)


In [21]:
images = {}
print(images)

for image_directory_path in list_image_directory_paths:
    image_class_name = os.path.basename(os.path.normpath(image_directory_path))
    print(image_class_name, image_directory_path)
    n = 0
    images[image_class_name] = []
    for image_file in glob.glob(image_directory_path + '*.png'):
        n += 1
        if n % 1000 == 0:
            print(len(images[image_class_name]), n)
        
        image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
        unseen_image = True
        for prev_image_sum, prev_image in images[image_class_name]:
            err = mse(image, prev_image)
            if err < 200:
                unseen_image = False
                break

        if unseen_image:
            images[image_class_name].append((image.sum(), image))
            
    print(len(images[image_class_name]), n)

{}
('0', '/Users/jacksimpson/Data/beeunique/output/drive8-merged/0/')
(421, 1000)
(646, 2000)
(790, 3000)
(924, 4000)
(1045, 5000)
(1153, 6000)
(1264, 7000)
(1362, 8000)
(1441, 9000)
(1520, 10000)
(1618, 11000)
(1690, 12000)
(1767, 13000)
(1834, 14000)
(1889, 15000)
(1914, 15468)
('1', '/Users/jacksimpson/Data/beeunique/output/drive8-merged/1/')
(350, 1000)
(558, 2000)
(665, 3000)
(771, 4000)
(880, 5000)
(966, 6000)
(1046, 7000)
(1111, 8000)
(1189, 9000)
(1254, 10000)
(1316, 11000)
(1371, 12000)
(1426, 13000)
(1468, 14000)
(1511, 15000)
(1555, 16000)
(1594, 16813)
('2', '/Users/jacksimpson/Data/beeunique/output/drive8-merged/2/')
(4, 14)
('3', '/Users/jacksimpson/Data/beeunique/output/drive8-merged/3/')
(257, 1000)
(390, 2000)
(493, 3000)
(568, 4000)
(636, 5000)
(694, 6000)
(756, 7000)
(824, 8000)
(889, 9000)
(943, 10000)
(986, 11000)
(1040, 12000)
(1080, 13000)
(1124, 14000)
(1185, 15000)
(1243, 16000)
(1282, 17000)
(1321, 18000)
(1348, 19000)
(1360, 19370)
('4', '/Users/jacksimpson/D

In [22]:
print(images.keys())

['CircleLine', 'heart', 'Note1', 'DD', 'Note2', 'Question', 'Pillars', 'HH', 'Ampersand', 'Plant', 'Triangle', 'nn', '0', 'Ankh', 'unknown', 'Trident', 'Asterisk', 'UU', '1', 'EE', '3', '2', '5', '4', '7', '6', '8', 'Omega', 'CircleCross', 'AA', 'SS', 'Peace', 'Tadpole', 'ArrowLine', 'PP', 'lines3', 'y', 'ZZ', 'radioactive', 'Umbrella', 'Dot', 'a', 'e', 'Power', 'KK', 'h', 'Queen', 'Plane', 'MM', 'r', 'circlehalf', 'w', 'necklace', 'Scissors']


In [23]:
n = 0
for tag in images.keys():
    tag_class_dir = '/Users/jacksimpson/Data/beeunique/training/drive8/{}'.format(tag)
    images[tag].sort(key=lambda x: x[0])
    os.makedirs(tag_class_dir)
    for im_sum, im in images[tag]:
        n += 1
        fname = tag_class_dir + '/' + str(n) + '_' + uuid.uuid4().hex + '.png'
        cv2.imwrite(fname, im)